In [28]:
import pandas as pd
# go to https://ford-parts.com.ru/bitrix/admin/iblock_data_export.php?lang=ru
# Каталог -> Каталог товаров и Торговые предложения
# Вытащить категории, сабкатегории и скрестить с новой версией в Postgres
#

In [29]:
catalogue = pd.read_csv("data/category.csv", sep=";")

In [30]:
catalogue = catalogue[
    [
        "IE_XML_ID",
        "IE_NAME",
        "IE_ID",
        "IE_PREVIEW_PICTURE",
        "IE_PREVIEW_TEXT",
        "IE_CODE",
        "IC_GROUP1",
        "IC_GROUP2",
    ]
]

In [31]:
catalogue = catalogue.rename(
    columns={
        "IE_XML_ID": "bitrix_id",
        "IE_NAME": "name",
        "IE_ID": "IE_ID",
        "IE_PREVIEW_PICTURE": "old_image",
        "IE_PREVIEW_TEXT": "cross_number",
        "IC_GROUP1": "category",
        "IC_GROUP2": "sub_category",
    }
)

In [32]:
catalogue

,bitrix_id,name,IE_ID,old_image,cross_number,IE_CODE,category,sub_category
0,F21-227,Ecosport 2018-> (2066235),247198,NaN,2066235,f21-227,NaN,NaN
1,F21-230,Фильтр вентиляции картерных газов (поролон) Fo...,247202,NaN,1045142,f21-230,NaN,NaN
2,F18-1775,Explorer 2011-> Направляющая переднего суппорта,250492,NaN,NaN,f18-1775,NaN,NaN
3,F18-1777,Mondeo 1996-2015 / S-MAX 2007- / Galaxy 1996-2...,250495,NaN,NaN,f18-1777,NaN,NaN
4,F18-1780,RANGER 2006- Направляющие супорта пер. (к-т из...,250499,NaN,3684757\n3684757\n3684757,f18-1780,NaN,NaN
...,...,...,...,...,...,...,...,...
9707,378466,Подвесной подшипник карданного вала MB Sprinte...,378466,/upload/iblock/2da/1u6636bocs45n1il52sk43yegf8...,NaN,podvesnoy_podshipnik_kardannogo_vala_mb_sprint...,Разные запчасти не для Ford,NaN
9708,378565,Радиатор ДГУ FG Wilson / Olympian с двиг. Perk...,378565,/upload/iblock/0da/zs35b9mlx202cio8h748sh38ley...,"2485B280, T430481, 2485b281, 301-05075\r\n",radiator_dgu_fg_wilson_olympian_s_dvig_perkins...,Разные запчасти не для Ford,NaN
9709,378582,Амортизатор задний MB Sprinter / VW LT 1996-20...,378582,/upload/iblock/662/hua3rfalrek50120s9z7nlzdtiq...,NaN,amortizator_zadniy_mb_sprinter_vw_lt_1996_2006...,Разные запчасти не для Ford,NaN
9710,378609,Прокладка клапанной крышки MB 602 mot. 71-2657...,378609,/upload/iblock/dfb/ly905grbm1ryo6s8m7011ylw1fx...,6020160321,prokladka_klapannoy_kryshki_mb_71_26574_10_vic...,Разные запчасти не для Ford,NaN


In [ ]:
category = catalogue["category"].drop_duplicates()
category

In [36]:
sub_category = catalogue["sub_category"].drop_duplicates()
sub_category

0                                                   NaN
1012                                   Механизмы замков
1072                               Полукольца коленвала
1093                                Автомобильный набор
1094                                   Актуатор Турбины
                             ...                       
9646                                Маслянный радиаторы
9667                           Жидкость стеклоомывателя
9670                                          Батарейки
9673    Датчик давления воздуха (MAP / наддув / выхлоп)
9685                               Датчики кондиционера
Name: sub_category, Length: 291, dtype: object

## Предложения (20 000 -> 25 000 за 4 месяца)
Скрестить с прайс-листом для цен и кол-ва

In [ ]:
offers = pd.read_csv("data/offers.csv", sep=",")

In [20]:
offers = offers[
    [
        "IE_XML_ID",
        "IE_NAME",
        "IE_ID",
        "IE_PREVIEW_PICTURE",
        "IE_PREVIEW_TEXT",
        "IP_PROP661",
        "IP_PROP660",
    ]
]

In [22]:
offers = offers.rename(
    columns={
        "IE_XML_ID": "offer_bitrix_id",
        "IE_NAME": "name",
        "IE_ID": "IE_ID",
        "IE_PREVIEW_PICTURE": "old_image",
        "IE_PREVIEW_TEXT": "internal_description",
        "IP_PROP661": "manufacturer_number",
        "IP_PROP660": "brand",
    }
)

In [37]:
offers

,offer_bitrix_id,name,IE_ID,old_image,internal_description,manufacturer_number,brand
0,F8-2,Focus II-III / C-Max / Kuga / Mondeo / S-Max 0...,238652,/upload/iblock/d72/24sqmsxa5hpfyif8n4t4k5w1taf...,NaN,EZCFR025,NTY
1,F8-3,Замок двери багажника Focus II-III,238654,/upload/iblock/7b9/5vsxhfn3ve3rfwr3xm5fhcy63sy...,NaN,ZV17698,ZZVF
2,F8-4,Focus II-III / C-Max / Kuga / Mondeo / S-Max 0...,238655,/upload/iblock/317/f1lpms9s3drhuewt5lyiwoyr1fd...,8M51 R442A66 DB,1856670,POLCAR
3,F8-5,Focus II-III / C-Max / Kuga / Mondeo / S-Max 0...,238656,/upload/iblock/317/f1lpms9s3drhuewt5lyiwoyr1fd...,NaN,ABL1859161,ABLTON
4,F8-6,Механизм замка Connect задней (высокой) двери ...,238658,NaN,NaN,1419513,FORD
...,...,...,...,...,...,...,...
25018,378656,Connect 2002-13 рессоры серьги наружный 54mm,378656,/upload/iblock/011/0suji2756a9hzd88uy40rts823f...,NaN,FD-BS292,KAUTEK
25019,378657,CONNECT 1.8 TDi LYNX / Fiesta 2002-> все модели,378657,/upload/iblock/165/1sh09jtoegigia1m843vdc4bck1...,NaN,13MOT0220,SIWAY
25020,378658,Втулка троса АКПП KV6Z7K340A / 2587456 / DG9...,378658,/upload/iblock/187/6cxd3w6kw430o9ep4gny0t7p87c...,NaN,5658329,Ford
25021,378660,Шпилька передняя Transit T-15 1991-2000 RH / LH,378660,/upload/iblock/730/y8ihja88r5ln212kfqsl2ylnw3g...,NaN,11-710-005,FASE
